<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-assets/phoenix/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>
<h1 align="center">Tracing and Evaluating a LlamaIndex OpenAI Agent Application</h1>

With the new OpenAI API that supports function calling, it’s never been easier to build your own agent.

In this notebook tutorial, we showcase how to write your own OpenAI agent in under 50 lines of code and use Phoenix to inspect the internals of the Agent. It is minimal, yet feature complete (with ability to carry on a conversation and use tools).

Install LlamaIndex and other dependencies.

In [ ]:
!pip install -qq arize-phoenix llama-index

Import libraries.

In [ ]:
import os
from getpass import getpass

import openai
import pandas as pd
import phoenix as px
from llama_index.agent import OpenAIAgent
from llama_index.callbacks import CallbackManager
from phoenix.trace.llama_index import (
    OpenInferenceTraceCallbackHandler,
)
from phoenix.trace.span_json_encoder import spans_to_jsonl

pd.set_option("display.max_colwidth", 1000)

## 2. Launch Phoenix

You can run Phoenix in the background to collect trace data emitted by any LlamaIndex application that has been instrumented with the `OpenInferenceTraceCallbackHandler`.

Launch Phoenix and follow the instructions in the cell output to open the Phoenix UI (the UI should be empty because we have yet to run a LlamaIndex application).

In [ ]:
session = px.launch_app()

Let’s start by importing some simple building blocks.

The main thing we need is:

- the OpenAI API (using our own llama_index LLM class)
- a place to keep conversation history
- a definition for tools that our agent can use.

Let’s define some very simple calculator tools for our agent.

In [ ]:
from llama_index.llms import OpenAI
from llama_index.tools import FunctionTool


def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)


def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)

In [ ]:
if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")
openai.api_key = openai_api_key
os.environ["OPENAI_API_KEY"] = openai_api_key

## Initialize the OpenAI Agent

Now, we define our agent that’s capable of holding a conversation and calling tools.

The meat of the agent logic is in the chat method. At a high-level, there are 3 steps:

- Call OpenAI to decide which tool (if any) to call and with what arguments.

- Call the tool with the arguments to obtain an output

- Call OpenAI to synthesize a response from the conversation context and the tool output.

The reset method simply resets the conversation context, so we can start another conversation.

For fun, let's make the agent chat in the style of Shakespeare.

In [ ]:
from llama_index.prompts.system import SHAKESPEARE_WRITING_ASSISTANT

llm = OpenAI(model="gpt-3.5-turbo-0613")
callback_handler = OpenInferenceTraceCallbackHandler()
callback_manager = CallbackManager(handlers=[callback_handler])
agent = OpenAIAgent.from_tools(
    [multiply_tool, add_tool],
     llm=llm, callback_manager=callback_manager,
     system_prompt=SHAKESPEARE_WRITING_ASSISTANT
)

Let's now chat with our agent!

In [ ]:
response = agent.query("What is (121 * 3) + 42?")
print(response)

Let's chat with our agent a few more times. This time with some follow-up questions.

In [ ]:
queries = [
    "What is (121 * 3) + 42?",
    "what is 3 * 3?",
    "what is 4 * 4?",
    "what is 75 * (3 + 4)?",
    "what is 23 times 87"
]

for query in queries:
    print(f"> {query}")
    response = agent.query(query)
    print(response)
    agent.reset()
    print("---")

We can now take a look at the traces in Phoenix. Note how the LLM spans contain the OpenAI function calls and we can inspect what tool the LLM is picking to run based on the queries.

To learn more about function calling, check out the [OpenAI API docs](https://openai.com/blog/function-calling-and-other-api-updates).


In [ ]:
session.url

We can also inspect the agent's chat history as a dataframe.

In [ ]:
ds = px.TraceDataset.from_spans(list(callback_handler.get_spans()))
ds.dataframe.head()

If you would like you can write the conversations to a file for later use.

In [ ]:
# Dump the contents to a file for safe keeping
export_trace = False
if export_trace:
    with open("trace.jsonl", "w") as f:
        f.write(spans_to_jsonl(callback_handler.get_spans()))